In [156]:
from mayavi import mlab


In [157]:
import tensorflow as tf
import numpy as np
import t3f
tf.set_random_seed(0)
np.random.seed(0)
import matplotlib.pyplot as plt
import metric_util as mt
import data_util as du
from t3f import shapes
from nilearn import image
from skimage.measure import compare_ssim as ssim
from tensorflow.python.util import nest
import copy
from nilearn import plotting
from t3f import ops
import mri_draw_utils as mrd
from t3f import initializers
from t3f import approximate
from scipy import optimize 
from scipy import special
from nilearn.image.resampling import coord_transform
from scipy import stats
from mayavi.mlab import *

In [158]:
subject_scan_path = du.get_full_path_subject1()
print "Subject Path: " + str(subject_scan_path)
x_true_org = mt.read_image_abs_path(subject_scan_path)

Subject Path: /work/pl/sch/analysis/data/COBRE001/swaAMAYER+cobre01_63001+M87100944+20110309at135133+RSTpre_V01_R01+CM.nii


In [159]:
data = np.array(image.index_img(x_true_org, 0).get_data())

In [160]:
x, y, z = np.mgrid[-78:78+1:3, -111:75+1:3, -51:84+1:3]

In [161]:
def get_i_j_k(x1, y1, z1):
    result = np.round(coord_transform(x1, y1, z1, np.linalg.inv(x_true_org.affine)))
    result = result.astype(int)
    result = (result[0], result[1], result[2])
    result = np.asarray(result).T
    return result[0], result[1],result [2]

In [162]:
def spatial_array(a, data, target):
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                #print i,j,k
                c1, c2, c3 = get_i_j_k(i,j,k)
                target[i,j,k] = data[c1,c2,c3]
    return target
    
    

In [163]:
target_array = np.zeros_like(data)


In [164]:
Fx = np.zeros_like(data)
Fy = np.zeros_like(data)
Fz = np.zeros_like(data)

In [165]:
Fx = spatial_array(x, data, Fx)

In [166]:
Fy = spatial_array(y, data, Fy)

In [167]:
Fz = spatial_array(z, data, Fz)

In [168]:
xx = data[0:53,:,:]

In [169]:
xx.shape

(53, 63, 46)

In [170]:
# Create a mayavi figure black on white
mlab.figure(bgcolor=(0., 0., 0.), fgcolor=(1., 1., 1.), size=(640, 480))

In [171]:
norm_data = np.linalg.norm(data)

In [172]:
mlab.clf()

In [173]:
Fnorm = np.sqrt(Fx**2 + Fy**2 + Fz**2)

In [174]:
Fmax = stats.scoreatpercentile(Fnorm.ravel(), 99)

In [175]:
Fx[Fnorm > Fmax] = Fmax * (Fx / Fnorm)[Fnorm > Fmax]
Fy[Fnorm > Fmax] = Fmax * (Fy / Fnorm)[Fnorm > Fmax]
Fz[Fnorm > Fmax] = Fmax * (Fz / Fnorm)[Fnorm > Fmax]
Fnorm[Fnorm > Fmax] = Fmax

In [176]:
field = mlab.pipeline.vector_field(x, y, z, Fx, Fy, Fz,
                                   #scalars = Fnorm,
                                   name='My field')

In [177]:
#src = mlab.pipeline.scalar_field(x, y, z,scalars = data)

In [178]:
vectors = mlab.pipeline.vectors(field,scale_factor=abs(x[0, 0, 0] - x[1, 1, 1]),
                                colormap='hsv')

In [179]:

mlab.axes()

# Mask 7 data points out of 8
vectors.glyph.mask_input_points = True
vectors.glyph.mask_points.on_ratio = 8

mlab.pipeline.vector_cut_plane(field, scale_factor=.1, colormap='jet')

# Add an iso_surface of the norm of the field
mlab.pipeline.iso_surface(mlab.pipeline.extract_vector_norm(field),
                          contours=[0.1*Fmax, 0.4*Fmax],
                          opacity=0.7, transparent=True)

#mlab.quiver3d(x, y, z, Fx, Fy, Fx, line_width=3, scale_factor=1)


#mlab.view(28, 84, 0.71)
mlab.show()


In [180]:
x.shape

(53, 63, 46)

In [181]:
#mlab.show()

In [182]:
np.gradient(Fx)

[array([[[    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ],
         [    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ],
         [    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ],
         ...,
         [    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ],
         [    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ],
         [    0.       ,     0.       ,     0.       , ...,
              0.       ,     0.       ,     0.       ]],
 
        [[ -270.53418  ,  -270.53418  ,  -170.99023  , ...,
             -7.8779297,    28.876953 ,    28.876953 ],
         [ -270.53418  ,  -270.53418  ,  -170.99023  , ...,
             -7.8779297,    28.876953 ,    28.876953 ],
         [ -249.07129  ,  -249.07129  ,  -154.2334   , 